import and visualization of the student database from UCI repo

In [284]:
from ucimlrepo import fetch_ucirepo 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn import model_selection

student_performance = fetch_ucirepo(id=320) 
 
X = student_performance.data.features 
y = student_performance.data.targets 

X.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,yes,no,no,4,3,4,1,1,3,4
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,yes,no,5,3,3,1,1,3,2
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,yes,no,4,3,2,2,3,3,6
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,yes,3,2,2,1,1,5,0
4,GP,F,16,U,GT3,T,3,3,other,other,...,yes,no,no,4,3,2,1,2,5,0


Extract and split data into train/test (80/20)

In [231]:
x_data=y[['G1','G2']]
y_data=y['G3']


Define the algorithm

In [232]:
def LinReg(x_train,y_train,x_test,y_test):
    reg=LinearRegression().fit(x_train,y_train)
    y_pred=reg.predict(x_test)
    results=r2_score(y_test,y_pred)
    return(results,reg.coef_,reg.intercept_)

Evaluation of best fit

In [248]:
best_acc=0
best_coef=[]
best_inter=0
for i in range(100):
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.30,random_state=50)
    Acc,coef,inter=LinReg(x_train,y_train,x_test,y_test)
    if Acc<best_acc:
        best_acc=Acc
        best_coef=coef
        best_inter=inter

print(Acc*100)

84.44583058727977


Preprocesing the data using MinMaxScaler

In [234]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(y)
y_sc=pd.DataFrame(scaler.transform(y),columns=['G1','G2','G3'])
y_sc.describe()

,G1,G2,G3
count,649.000000,649.000000,649.000000
mean,0.599951,0.608953,0.626632
std,0.144488,0.153349,0.170035
min,0.000000,0.000000,0.000000
25%,0.526316,0.526316,0.526316
50%,0.578947,0.578947,0.631579
75%,0.684211,0.684211,0.736842
max,1.000000,1.000000,1.000000


Spliting 80/20 after the normalization

In [240]:
x_datasc=y_sc[['G1','G2']]
y_datasc=y_sc['G3']


In [249]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.30,random_state=50)
Acc,coef,inter=LinReg(x_train,y_train,x_test,y_test)
print(Acc*100)

84.44583058727977


3rd Try: add the Absences column and standarize the data

In [371]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

X = student_performance.data.features 
data = student_performance.data.targets 
data=data.join(X['absences'])
data=data.join(X['studytime'])
data=data.join(X['freetime'])
data=data.join(X['failures'])
data=data.join(X['Dalc'])
le = LabelEncoder()
le.fit(X["internet"])
temp=pd.DataFrame(le.transform(X["internet"]),columns=["Internet"])
data=data.join(temp["Internet"])
le.fit(X["paid"])
temp=pd.DataFrame(le.transform(X["paid"]),columns=["paid"])
data=data.join(temp["paid"])
#data=scaler.fit_transform(data)
Scaler=StandardScaler()
Scaler.fit(data)
data=Scaler.fit_transform(data)
data=pd.DataFrame(data,columns=['G1','G2','G3','absences','ST','FT','F',"Dalc","internet",'paid'])
data.describe().round(2)

,G1,G2,G3,absences,ST,FT,F,Dalc,internet,paid
count,649.00,649.00,649.00,649.00,649.00,649.00,649.00,649.00,649.00,649.00
mean,0.00,-0.00,0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,0.00
std,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
min,-4.16,-3.97,-3.69,-0.79,-1.12,-2.08,-0.37,-0.54,-1.82,-0.25
25%,-0.51,-0.54,-0.59,-0.79,-1.12,-0.17,-0.37,-0.54,0.55,-0.25
50%,-0.15,-0.20,0.03,-0.36,0.08,-0.17,-0.37,-0.54,0.55,-0.25
75%,0.58,0.49,0.65,0.50,0.08,0.78,-0.37,0.54,0.55,-0.25
max,2.77,2.55,2.20,6.11,2.50,1.73,4.69,3.78,0.55,3.95


In [372]:
x=data[['G1','G2','absences','ST','FT','F','Dalc','internet','paid']]
y=data['G3']


In [373]:
y=data['G3']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30,random_state=50)
Acc,coef,inter=LinReg(x_train,y_train,x_test,y_test)
print(Acc*100)

84.35795927981293
